In [ ]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

#declare file path for cities data
cities_path = "output_data/cities.csv"

In [ ]:
cities_df = pd.read_csv(cities_path)
cities_df.dropna()
cities_df

In [ ]:
locations = cities_df[["Lat", "Lng"]].astype(float)
locations

In [ ]:
humidity = cities_df["Humidity"].tolist()
print(humidity)

In [ ]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

In [ ]:
hotel_df = cities_df.copy()

In [ ]:
hotel_df = hotel_df[hotel_df['Max Temp'] > 70]
hotel_df = hotel_df[hotel_df['Max Temp'] < 80]
hotel_df = hotel_df[hotel_df['Cloudiness'] > 20]
hotel_df = hotel_df[hotel_df['Cloudiness'] < 35]
hotel_df = hotel_df[hotel_df['Wind Speed'] < 10 ]
hotel_df = hotel_df[hotel_df['Humidity'] < 80 ]
hotel_df['Hotel'] = ""
hotel_df

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    #"location": (f"{hotel_df[index]},{hotel_df[index]}")  
    "rankby": "distance",
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    city = row['City']
    latitude = row['Lat']
    longitude = row['Lng']

    # add keyword to params dict
    params['location'] = (f"{latitude},{longitude}")

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {city} Hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel"], inplace=True)
hotel_df

In [ ]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = cities_df[["Lat", "Lng"]]

In [ ]:
hotel_locations = hotel_df[["Lat", "Lng"]].astype(float)

In [ ]:
fig = gmaps.figure()
markers = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

In [ ]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig